<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Initialization" data-toc-modified-id="Initialization-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Initialization</a></span></li><li><span><a href="#Testing" data-toc-modified-id="Testing-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Testing</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Equations" data-toc-modified-id="Equations-2.0.1"><span class="toc-item-num">2.0.1&nbsp;&nbsp;</span>Equations</a></span></li><li><span><a href="#Data-Manipulation" data-toc-modified-id="Data-Manipulation-2.0.2"><span class="toc-item-num">2.0.2&nbsp;&nbsp;</span>Data Manipulation</a></span></li></ul></li></ul></li><li><span><a href="#R-Stuff" data-toc-modified-id="R-Stuff-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>R Stuff</a></span></li></ul></div>

# Initialization

In [1]:
#from sympy.interactive.printing import init_printing
#init_printing(use_unicode=False, wrap_line=True, no_global=True)
from sympy import *
import numpy as np, pandas as pd
import sys, os
import xlrd
from itertools import permutations, product

<h5> Note: missing domestic & cross-border credit </h5>

In [2]:
scriptDir = os.path.dirname(os.path.realpath(sys.argv[0]))   # get script directory
genDir = "C:/Users/benmo/OneDrive/GitHub/DataPlayground" #get general functions path

os.chdir(genDir)

from General import * # import custom 'general' functions

os.chdir(scriptDir)

fpath = MyComp.financePath  #path to financial econ folder


cntryBook = xlrd.open_workbook(fpath + "/data/gvar/Country Data (1979Q1-2011Q2).xlsx")
sheets = cntryBook.sheet_names()

# Testing

In [ ]:
#disc_rate_sym, T_sym, FaceV_sym, price_sym = symbols('r T F P')
#C_sym = MatrixSymbol('C', 5, 1)

### Equations

### Data Manipulation 

In [4]:
data = pd.DataFrame([])
idx = pd.IndexSlice

for sheet in sheets:
    temp = pd.read_excel(fpath + "/data/gvar/Country Data (1979Q1-2011Q2).xlsx", sheet_name=sheet)
    temp['Country'] = sheet
    temp.set_index([temp['Country'], temp['date']], inplace=True)
    data = data.append(temp)
    
data = data.sort_index()
    

In [5]:
tradeMatrix = pd.read_excel(fpath + "/data/gvar/Trade Matrices (1980-2010).xls")
cntryCodes = pd.read_excel(fpath + "/data/gvar/Country Codes.xls", index_col='Country')

tradeMatrix['country_primary'] = tradeMatrix['country_primary'].apply(lambda x, code=cntryCodes: 
                                     code[code.values == x].index[0])

countries = tradeMatrix[tradeMatrix.year == 1980]['country_primary'].tolist()
tradeMatrix.rename(columns=lambda x, names=cntryCodes: 
                   names[names.values == int(x[1:])].index[0] if [x[1:]] in names
                   .values.astype(str).tolist() else x, 
                   inplace=True)



W = tradeMatrix.set_index([tradeMatrix.country_primary, 
                           pd.to_datetime(tradeMatrix.astype(str).year)])

wEurope = ['Spain', 'Belgium', 'Netherlands', 'Switzerland']
mEast = ['Saudi Arabia', 'Turkey']
seAsia = ['Korea', 'Thailand', 'Indonesia', 'Malaysia', 'Philippines', 'Singapore']
latAmerica = ['Argentina', 'Brazil', 'Chile', 'Peru', 'Mexico']

regionDict= {}
regionDict['Western Europe'] = wEurope
regionDict['Middle East'] = mEast
regionDict['Southeast Asia'] = seAsia
regionDict['Latin America'] = latAmerica


countriesOld = ['USA', 'UK', 'Germany', 'France', 'Italy', 'China', 'Japan']
countriesOld.extend(wEurope)
countriesOld.extend(mEast)
countriesOld.extend(seAsia)
countriesOld.extend(latAmerica)


regions = ['USA', 'UK', 'Germany', 'France', 'Italy', 'China', 'Japan']
regions.extend(['Western Europe','Middle East', 'Southeast Asia', 'Latin America'])


W = W.append(pd.DataFrame(index=list(product(['Western Europe','Middle East',
                                         'Southeast Asia', 'Latin America'], 
                                         list(map(str,range(1980,2011)))))))

for key, val in regionDict.items():
    W[key] = W[list(filter(lambda x: x in val, 
                                        W.columns.tolist()))].apply(sum, axis=1)
    for country in countries:
        W.loc[idx[key,:], country] = W[key][country].values
    
for key, val in regionDict.items():    
    for region in list(filter(lambda x: x!= key, regions[-4:])):
        W.loc[idx[key,:], region] = W.loc[idx[key,:],
                                          regionDict[region]].apply(sum, axis=1)    

    
W = W.sort_index()
idx = pd.IndexSlice


Wold = W[regions]

Wold = Wold.apply(lambda x: x/x.sum(), axis=1)


In [ ]:
regions[-4:]

In [6]:


Wold.loc[idx['Western Europe', '1996':'1998'], :].iloc[0]

USA               0.101464
UK                0.132623
Germany           0.309007
France            0.189977
Italy             0.110223
China             0.013812
Japan             0.039851
Western Europe         NaN
Middle East       0.020099
Southeast Asia    0.053010
Latin America     0.029935
Name: (Western Europe, 1996-01-01 00:00:00), dtype: float64

In [8]:
W = pd.DataFrame(index=regions, columns=regions) 

for region in regions:
    W[region] = Wold.loc[idx[region, '1996':'1998'], :].apply(np.mean, axis=0)
    
W

In [ ]:

for key, val in regionDict.items(): 
    print([key, val])


# R Stuff

In [ ]:
fpath


In [ ]:
cntryCodes[cntryCodes.values == 160]